In [1]:
from pathlib import Path
from glob import glob

In [3]:
data_adr = Path(r"/Users/simonvanstaalduine/AMOLF-SHIMIZU Dropbox/Simon van Staalduine")
plate_id = r"20250122_Plate017/Img"
video_id = r"032"
frame_id1 = r"Img/Img_00000.tif"
frame_id2 = r"Img/Img_00001.tif"
total_path = data_adr / video_id




In [4]:
print(total_path)

/Users/simonvanstaalduine/AMOLF-SHIMIZU Dropbox/Simon van Staalduine/032
